# Lesson 2: Sequential Chats and Customer Onboarding

## Setup

In [50]:
#import libs
import os
import pprint
from autogen import ConversableAgent, UserProxyAgent
from utils import get_azure_openai_config
import groq

In [51]:
azure_config = get_azure_openai_config()

config_list = [
    {
        "model": "gemma2:2b",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
        "temperature":2
    },{
        "model": "llama3.2:3b",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
        "temperature":2
    },{
        "model": "llama3-70b-8192",
        "base_url": "https://api.groq.com/openai/v1",  
        "api_key": os.getenv("GROQ_API_KEY"),
        "api_type": "groq",
        "temperature":1 
    },{
        "model": azure_config["deployment_name"],
        "api_key": azure_config["api_key"],
        "api_type": "azure",
        "api_version": azure_config["api_version"],
        "temperature":2
    }
]

llm_config = {"config_list":config_list}

In [52]:
llm_config['config_list'][2]

{'model': 'llama3-70b-8192',
 'base_url': 'https://api.groq.com/openai/v1',
 'api_key': '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',
 'api_type': 'groq',
 'temperature': 1}

# Construindo os agentes

In [53]:
# The Number Agent always returns the same numbers.
number_agent = ConversableAgent(
    name="Number_Agent",
    system_message="You return me the numbers I give you, one number each line.",
    llm_config=llm_config['config_list'][2],
    human_input_mode="NEVER",
)

# The Adder Agent adds 1 to each number it receives.
adder_agent = ConversableAgent(
    name="Adder_Agent",
    system_message="You add 1 to each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config['config_list'][2],
    human_input_mode="NEVER",
)

# The Multiplier Agent multiplies each number it receives by 2.
multiplier_agent = ConversableAgent(
    name="Multiplier_Agent",
    system_message="You multiply each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config['config_list'][2],
    human_input_mode="NEVER",
)

# The Subtracter Agent subtracts 1 from each number it receives.
subtracter_agent = ConversableAgent(
    name="Subtracter_Agent",
    system_message="You subtract 1 from each number I give you and return me the new numbers, one number each line.",
    llm_config=llm_config['config_list'][2],
    human_input_mode="NEVER",
)

# The Divider Agent divides each number it receives by 2.
divider_agent = ConversableAgent(
    name="Divider_Agent",
    system_message="You divide each number I give you by 2 and return me the new numbers, one number each line.",
    llm_config=llm_config['config_list'][2],
    human_input_mode="NEVER",
)

In [54]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = number_agent.initiate_chats(
    [
        {
            "recipient": adder_agent,
            "message": "14",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": multiplier_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": subtracter_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
        {
            "recipient": divider_agent,
            "message": "These are my numbers",
            "max_turns": 2,
            "summary_method": "last_msg",
        },
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Adder_Agent):

14

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

15

--------------------------------------------------------------------------------
Number_Agent (to Adder_Agent):

15

--------------------------------------------------------------------------------
Adder_Agent (to Number_Agent):

16

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Number_Agent (to Multiplier_Agent):

These are my numbers
Context: 
16

--------------------------------------------------------------------------------
Mul

In [55]:
print("First Chat Summary: ", chat_results[0].summary)
print("Second Chat Summary: ", chat_results[1].summary)
print("Third Chat Summary: ", chat_results[2].summary)
print("Fourth Chat Summary: ", chat_results[3].summary)

First Chat Summary:  16
Second Chat Summary:  64
Third Chat Summary:  Here are the results of subtracting 1 from each of your numbers:

14
62
Fourth Chat Summary:  I'll divide each of the numbers by 2. Here are the results:

3.5
15.5


In [18]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding_Personal_Information_Agent",
    system_message='''Você é um agente de onboarding de clientes, 
                    e está aqui para ajudar novos clientes a começar a usar nosso produto. 
                    Seu trabalho é coletar o nome e a localização do cliente. 
                    Não peça outras informações. 
                    Retorne 'TERMINAR' quando tiver coletado todas as informações.''',
    llm_config=llm_config['config_list'][2],
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [19]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding_Topic_preference_Agent",
    system_message='''Você é um agente de onboarding de clientes, 
                    e está aqui para ajudar novos clientes a começar a usar nosso produto. 
                    Seu trabalho é coletar as preferências de tópicos de notícias do cliente. 
                    Não peça outras informações. Retorne 'TERMINAR' quando tiver coletado todas as informações.''',
    llm_config=llm_config['config_list'][2],
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [32]:
customer_engagement_agent = ConversableAgent(
    name="Customer_Engagement_Agent",
    system_message='''Você é um agente de atendimento ao cliente prestativo, 
                    e está aqui para proporcionar diversão ao cliente com base 
                    nas informações pessoais e preferências de {'topic': ''} do usuário. 
                    Isso pode incluir curiosidades, piadas ou histórias interessantes. 
                    Certifique-se de tornar isso envolvente e divertido! 
                    Retorne 'TERMINAR' quando tiver terminado.''',
    llm_config=llm_config['config_list'][2],
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINAR" in msg.get("content").lower(),
)

In [41]:
customer_proxy_agent = UserProxyAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "TERMINAR" in msg.get("content").lower(),
)

## Creating tasks

Now, you can craft a series of tasks to facilitate the onboarding process.

In [38]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Olá, estou aqui para te ajudar a começar a usar nosso produto."
            "Você poderia me dizer seu nome e localização?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Retorne as informações do cliente "
                             "como um objeto JSON apenas: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": "Ótimo! Você poderia me dizer sobre quais tópicos tem interesse em ler?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Retorne as informações do cliente "
                             "como um objeto JSON apenas: "
                             "{'Context: ': ''}",
        },
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Vamos encontrar algo divertido para ler.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
        "summary_prompt" : "Passe para o agente customer_engagement_agent "
                             " o contexto vindo do json do agente onboarding_topic_preference_agent",
        },
]

## Iniciando o chat

**Nota**: Você pode receber uma resposta ligeiramente diferente da que é mostrada no vídeo. Sinta-se à vontade para tentar diferentes entradas, como nome, localização e preferências.

In [42]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Olá, estou aqui para te ajudar a começar a usar nosso produto.Você poderia me dizer seu nome e localização?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

joao

--------------------------------------------------------------------------------
Onboarding_Personal_Information_Agent (to customer_proxy_agent):

Olá Joao! Você poderia me dizer sua localização?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding_Personal_Information_Agent):

sampa

--------------------------------------------------------------------------------

***************************************************

## Print out the summary

In [43]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'content': '', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}


{'content': '', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}


{'content': '', 'refusal': None, 'role': 'assistant', 'function_call': None, 'tool_calls': None}




## Print out the cost

In [44]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.0010756700000000001, 'llama3-70b-8192': {'cost': 0.0010756700000000001, 'prompt_tokens': 1625, 'completion_tokens': 148, 'total_tokens': 1773}}, 'usage_excluding_cached_inference': {'total_cost': 0.0005920299999999999, 'llama3-70b-8192': {'cost': 0.0005920299999999999, 'prompt_tokens': 899, 'completion_tokens': 78, 'total_tokens': 977}}}


{'usage_including_cached_inference': {'total_cost': 0.0007248100000000001, 'llama3-70b-8192': {'cost': 0.0007248100000000001, 'prompt_tokens': 1191, 'completion_tokens': 28, 'total_tokens': 1219}}, 'usage_excluding_cached_inference': {'total_cost': 0.0007248100000000001, 'llama3-70b-8192': {'cost': 0.0007248100000000001, 'prompt_tokens': 1191, 'completion_tokens': 28, 'total_tokens': 1219}}}


{'usage_including_cached_inference': {'total_cost': 0.0010173, 'llama3-70b-8192': {'cost': 0.0010173, 'prompt_tokens': 1115, 'completion_tokens': 455, 'total_tokens': 1570}}, 'usage_excluding_cached_inferenc